# Lab 5 - Joining Uber Pick-Ups, Stations, and Boroughs

In [Lecture 3.4](./3_4_joining_large_and_small_files.ipynb), we introduced using a Python `dict` to join a large and small table.  In this lab, we will practice this technique on the uber data set.

**Note:** Make sure that you download and unzip the file `Uber-Jan-Feb-FOIL.csv` before proceeding.

In [1]:
import pandas as pd
from dfply import *

In [2]:
c_size = 500000
trips = pd.read_csv("./data/uber/uber-trip-data/uber-raw-data-janjune-15.csv", chunksize=c_size)
first_chunk = next(trips)

In [3]:
first_chunk.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
0,B02617,2015-05-17 09:47:00,B02617,141
1,B02617,2015-05-17 09:47:00,B02617,65
2,B02617,2015-05-17 09:47:00,B02617,100
3,B02617,2015-05-17 09:47:00,B02774,80
4,B02617,2015-05-17 09:47:00,B02617,90


## <font color="red"> Problem 1 - Creating 2 Location `dict`</font>

Read the file `taxi-zone-lookup.csv` and inspect the columns. What 2 pieces of information will this file allow us to add to the `uber-raw-data-janjune-15.csv`?  Make a `dict` for each of these variables.

> *Borough and Zone*


In [4]:
# Your code here

taxiZone = pd.read_csv('./data/uber/uber-trip-data/taxi-zone-lookup.csv')

In [5]:
taxiZone.LocationID

0        1
1        2
2        3
3        4
4        5
5        6
6        7
7        8
8        9
9       10
10      11
11      12
12      13
13      14
14      15
15      16
16      17
17      18
18      19
19      20
20      21
21      22
22      23
23      24
24      25
25      26
26      27
27      28
28      29
29      30
      ... 
235    236
236    237
237    238
238    239
239    240
240    241
241    242
242    243
243    244
244    245
245    246
246    247
247    248
248    249
249    250
250    251
251    252
252    253
253    254
254    255
255    256
256    257
257    258
258    259
259    260
260    261
261    262
262    263
263    264
264    265
Name: LocationID, Length: 265, dtype: int64

In [ ]:
locationID = first_chunk.locationID

In [ ]:
taxiZone.head()

## <font color="red"> Problem 2 - Dispatch Translation</font>

The following table was taken from the FiveThirtyEight github page and contains the names of each Uber dispatch station.  Create a translation `dict` for these data.

Base Code | Base Name
---|---------
B02512 | Unter
B02598 | Hinter
B02617 | Weiter
B02682 | Schmecken
B02764 | Danach-NY
B02765 | Grun
B02835 | Dreist
B02836 | Drinnen

In [ ]:
# Your code he

taxiLocationID = taxiZone.LocationID
taxiBorough = taxiZone.Borough
ZoneID = taxiZone.Zone

In [ ]:
#convert to list comprehension. 

taxiLocationID = [data for data in taxiLocationID]
taxiBorough = [data for data in taxiBorough]
taxiZone = [data for data in taxiZone]

In [ ]:
Bo = {code:name for code,name in zip(taxiLocationID,taxiBorough)}

In [ ]:
Zo = {code:name for code,name in zip(taxiLocationID,ZoneID)}

In [ ]:
Borough = ['B02512','B02598','B02617','B02682','B02764','B02765', 'B02835', 'B02836']
BaseName = ['Unter','Hinter','Weiter','Schmecken','Danach-NY','Grun','Dreist','Drinnen']

base = {code:name for code,name in zip(Borough, BaseName)}

In [ ]:
base

In [ ]:
#add_Borough = lambda df: (df
                           >> mutate(base_name = X.LocationID.replace(to_replace=base)))



first_chunk = add_base_name(first_chunk).head()

In [ ]:
first_chunk

In [ ]:
add_base_name = lambda df: (df
                           >> mutate(base_name = X.Affiliated_base_num.replace(to_replace=base)))
first_chunk = add_base_name(first_chunk).head()

In [ ]:
boroughDict = {iD:name for iD, name in zip(taxiZone.LocationID, taxiZone.Borough)}
baseDict = {iD:name for iD, name in zip(taxiZone.LocationID, taxiZone.Zone)}


In [ ]:
add_base_name = lambda df: (df
                           >> mutate(base_name = X.locationID.replace(to_replace=boroughDict)))
first_chunk = add_base_name(first_chunk).head()

## <font color="red"> Problem 3 - Prototyping a Helper Function</font>

Use the first chunk to prototype a helper function that

1. Add three new columns, one for each key/translation.
2. Drop each of the associated keys.
3. Convert the `pickup_date` to a datetime column.
4. Add various datepart columns

This function should use appropriate `dfply` functions and a pipe.

In [ ]:
pickupDate = first_chunk.Pickup_date


In [ ]:
# converted to Datetime. 

import datetime
pickupDate = pd.to_datetime(pickupDate)
pickupDate[0]

## <font color="red"> Problem 4 - Creating a SQL database</font>

Use `pandas` and your helper function to create a `sqlite` database for this file.

## <font color="red"> Problem 5 - Exploring the pickups</font>

**Question of Interest:** I am interested in difference between pick-ups in terms of both time and Borough.  Use aggregation and visualizations to construct a group that illustrates an interesting difference between the Boroughs.